In [153]:
# Projeto: Validador de Data Quality (Datas)
#
# Este projeto implementa um pipeline de Validação e Tratamento de Qualidade de Dados, focado na coluna de datas.
# O fluxo de trabalho segue as seguintes etapas:
#
# 1. Carregamento e Inspeção Inicial do Dataset.
# 2. Diagnóstico do Formato e Identificação de Missing Values.
# 3. Tratamento Híbrido: Padronização da coluna de datas utilizando a função 'parse_data_hibrido'.
# 4. Validação das Regras de Negócio, incluindo:
#    - Checagem de Datas Futuras.
#    - Checagem de Datas Antigas (limite < 1900).
# 5. Geração e Apresentação do Relatório Final de Status da Qualidade.


In [154]:
import pandas as pd
import numpy as np
from datetime import datetime
import dateparser

In [155]:
df = pd.read_csv("arq.csv")
df.head()

,Data Compra
0,2024-05-20
1,20/05/2024
2,"maio 20, 2024"
3,2024/05/20
4,05-20-2024


In [156]:
coluna = "Data Compra"

In [157]:
datas_convertidas = pd.to_datetime(df[coluna], format="%Y-%m-%d", errors="coerce")
datas_convertidas




,Data Compra
0,2024-05-20
1,NaT
2,NaT
3,NaT
4,NaT
5,NaT
6,NaT
7,NaT
8,NaT
9,NaT


In [158]:
df["Status_Formato"] = datas_convertidas.isna()
df

,Data Compra,Status_Formato
0,2024-05-20,False
1,20/05/2024,True
2,"maio 20, 2024",True
3,2024/05/20,True
4,05-20-2024,True
5,25 fev. 2023,True
6,1/1/2025 15:30:00,True
7,28 de Novembro de 2023,True
8,20 dez. 2024,True
9,20230715,True


In [159]:
total_erros = df["Status_Formato"].sum()
total_erros


np.int64(9)

In [160]:
total_registros = df.shape[0]
total_registros


10

In [161]:
total_validas = total_registros - total_erros
percentual_erro = (total_erros / total_registros) * 100

total_validas, percentual_erro


(np.int64(1), np.float64(90.0))

In [162]:
limite = 10  # limite máximo de erro permitido (%)

if percentual_erro > limite:
    status = "REPROVADO"
else:
    status = "APROVADO"

status


'REPROVADO'

In [163]:
df.isnull().sum()


,0
Data Compra,0
Status_Formato,0


In [164]:
df.head()


,Data Compra,Status_Formato
0,2024-05-20,False
1,20/05/2024,True
2,"maio 20, 2024",True
3,2024/05/20,True
4,05-20-2024,True


In [165]:
# MÓDULO 2 — Missing Values

missing_por_coluna = df.isnull().sum()
total_missing = missing_por_coluna.sum()

if total_missing == 0:
    status_missing = "APROVADO"
else:
    status_missing = "REPROVADO"

print("VALIDAÇÃO — MISSING VALUES")
print("---------------------------")
print(missing_por_coluna)
print(f"Total de valores vazios: {total_missing}")
print(f"STATUS: {status_missing}")


VALIDAÇÃO — MISSING VALUES
---------------------------
Data Compra       0
Status_Formato    0
dtype: int64
Total de valores vazios: 0
STATUS: APROVADO


In [166]:
# MÓDULO 2 — Missing Values

missing_por_coluna = df.isnull().sum()
total_missing = missing_por_coluna.sum()

if total_missing == 0:
    status_missing = "APROVADO"
else:
    status_missing = "REPROVADO"

print("VALIDAÇÃO — MISSING VALUES")
print("---------------------------")
print(missing_por_coluna)
print(f"Total de valores vazios: {total_missing}")
print(f"STATUS: {status_missing}")


VALIDAÇÃO — MISSING VALUES
---------------------------
Data Compra       0
Status_Formato    0
dtype: int64
Total de valores vazios: 0
STATUS: APROVADO


In [167]:
import os
os.listdir()

['.config', '.ipynb_checkpoints', 'arq.csv', 'sample_data']

In [168]:
df = pd.read_csv("arq.csv")
df.head()


,Data Compra
0,2024-05-20
1,20/05/2024
2,"maio 20, 2024"
3,2024/05/20
4,05-20-2024


In [169]:
df.dtypes


,0
Data Compra,object


In [170]:
# MÓDULO 3 — Tipos de Dados

tipos = df.dtypes

# Regra: 'Data Compra' deve ser datetime64
tipo_correto = "datetime64[ns]"
tipo_atual = str(df["Data Compra"].dtype)

if tipo_atual == tipo_correto:
    status_tipos = "APROVADO"
else:
    status_tipos = "REPROVADO"

print("VALIDAÇÃO — TIPOS DE DADOS")
print("----------------------------")
print(tipos)
print(f"Tipo atual da coluna Data Compra: {tipo_atual}")
print(f"Tipo esperado: {tipo_correto}")
print(f"STATUS: {status_tipos}")


VALIDAÇÃO — TIPOS DE DADOS
----------------------------
Data Compra    object
dtype: object
Tipo atual da coluna Data Compra: object
Tipo esperado: datetime64[ns]
STATUS: REPROVADO


In [171]:
df["Data Compra"] = pd.to_datetime(df["Data Compra"], errors="coerce")
df["Data Compra"].head()


,Data Compra
0,2024-05-20
1,NaT
2,NaT
3,NaT
4,NaT


In [172]:
from datetime import datetime

data_hoje = datetime.today()
data_minima = datetime(1900, 1, 1)


In [173]:
from datetime import datetime

data_hoje = datetime.today()

df["Erro_Data_Futura"] = df["Data Compra"] > data_hoje
df["Erro_Data_Futura"].head()


,Erro_Data_Futura
0,False
1,False
2,False
3,False
4,False


In [174]:
from datetime import datetime

data_minima = datetime(1900, 1, 1)

df["Erro_Data_Antiga"] = df["Data Compra"] < data_minima
df["Erro_Data_Antiga"].head()


,Erro_Data_Antiga
0,False
1,False
2,False
3,False
4,False


In [175]:
df["Erro_Data_Antiga"]


,Erro_Data_Antiga
0,False
1,False
2,False
3,False
4,False
5,False
6,False
7,False
8,False
9,False


In [176]:
# MÓDULO 4 — Regras de Negócio

erros_futuro = df["Erro_Data_Futura"].sum()
erros_antiga = df["Erro_Data_Antiga"].sum()
total_erros_regras = erros_futuro + erros_antiga

if total_erros_regras == 0:
    status_regras = "APROVADO"
else:
    status_regras = "REPROVADO"

print("VALIDAÇÃO — REGRAS DE NEGÓCIO")
print("-------------------------------")
print(f"Erros: Data no futuro = {erros_futuro}")
print(f"Erros: Data muito antiga (< 1900) = {erros_antiga}")
print(f"Total de erros de regras: {total_erros_regras}")
print(f"STATUS: {status_regras}")


VALIDAÇÃO — REGRAS DE NEGÓCIO
-------------------------------
Erros: Data no futuro = 0
Erros: Data muito antiga (< 1900) = 0
Total de erros de regras: 0
STATUS: APROVADO


In [177]:
datas_convertidas = pd.to_datetime(df["Data Compra"], format="%Y-%m-%d", errors="coerce")

df["Status_Formato"] = datas_convertidas.isna()
df["Status_Formato"].head()


,Status_Formato
0,False
1,True
2,True
3,True
4,True


In [178]:
print("=======================================")
print("       RELATÓRIO FINAL DE QUALIDADE    ")
print("=======================================\n")

# 1. FORMATO
erros_formato = df["Status_Formato"].sum()
if erros_formato == 0:
    status_formato = "APROVADO"
else:
    status_formato = "REPROVADO"

print("1) Validação do Formato")
print("------------------------")
print(f"Total de erros de formato: {erros_formato}")
print(f"STATUS: {status_formato}\n")

# 2. MISSING VALUES
missing_total = df.isnull().sum().sum()
if missing_total == 0:
    status_missing = "APROVADO"
else:
    status_missing = "REPROVADO"

print("2) Missing Values")
print("-------------------")
print(df.isnull().sum())
print(f"Total de missing values: {missing_total}")
print(f"STATUS: {status_missing}\n")

# 3. TIPOS
# Aqui verificamos se Data Compra está em datetime
if str(df["Data Compra"].dtype) == "datetime64[ns]":
    status_tipo = "APROVADO"
else:
    status_tipo = "REPROVADO"

print("3) Tipos de Dados")
print("-------------------")
print(f"Tipo de Data Compra: {df['Data Compra'].dtype}")
print(f"STATUS: {status_tipo}\n")

# 4. REGRAS DE NEGÓCIO
erros_futuro = df["Erro_Data_Futura"].sum()
erros_antiga = df["Erro_Data_Antiga"].sum()
total_regras = erros_futuro + erros_antiga

if total_regras == 0:
    status_regras = "APROVADO"
else:
    status_regras = "REPROVADO"

print("4) Regras de Negócio")
print("-----------------------")
print(f"Erros: Data futura = {erros_futuro}")
print(f"Erros: Data antiga (<1900) = {erros_antiga}")
print(f"Total: {total_regras}")
print(f"STATUS: {status_regras}\n")

# STATUS GERAL
if (status_formato == "APROVADO" and
    status_missing == "APROVADO" and
    status_tipo == "APROVADO" and
    status_regras == "APROVADO"):
    status_geral = "APROVADO"
else:
    status_geral = "REPROVADO"

print("=======================================")
print("           STATUS GERAL: ", status_geral)
print("=======================================")


       RELATÓRIO FINAL DE QUALIDADE    

1) Validação do Formato
------------------------
Total de erros de formato: 9
STATUS: REPROVADO

2) Missing Values
-------------------
Data Compra         9
Erro_Data_Futura    0
Erro_Data_Antiga    0
Status_Formato      0
dtype: int64
Total de missing values: 9
STATUS: REPROVADO

3) Tipos de Dados
-------------------
Tipo de Data Compra: datetime64[ns]
STATUS: APROVADO

4) Regras de Negócio
-----------------------
Erros: Data futura = 0
Erros: Data antiga (<1900) = 0
Total: 0
STATUS: APROVADO

           STATUS GERAL:  REPROVADO


In [180]:
df = pd.read_csv("arq.csv")
df.head()


,Data Compra
0,2024-05-20
1,20/05/2024
2,"maio 20, 2024"
3,2024/05/20
4,05-20-2024


In [181]:
def parse_data_hibrido(texto):

    # 1. Formatos fixos
    formatos = [
        "%Y-%m-%d",
        "%d/%m/%Y",
        "%Y/%m/%d",
        "%m-%d-%Y",
        "%Y%m%d"
    ]

    for f in formatos:
        try:
            return datetime.strptime(texto, f)
        except:
            pass

    # 2. dateparser
    data_dp = dateparser.parse(texto, languages=["pt"])
    if data_dp:
        return data_dp

    # 3. Se nada funcionar
    return np.nan


In [182]:
df["Data Compra"] = df["Data Compra"].astype(str)
df["Data_Tratada"] = df["Data Compra"].apply(parse_data_hibrido)

df.head(10)


,Data Compra,Data_Tratada
0,2024-05-20,2024-05-20 00:00:00
1,20/05/2024,2024-05-20 00:00:00
2,"maio 20, 2024",2024-05-20 00:00:00
3,2024/05/20,2024-05-20 00:00:00
4,05-20-2024,2024-05-20 00:00:00
5,25 fev. 2023,2023-02-25 00:00:00
6,1/1/2025 15:30:00,2025-01-01 15:30:00
7,28 de Novembro de 2023,2023-11-28 00:00:00
8,20 dez. 2024,2024-12-20 00:00:00
9,20230715,2023-07-15 00:00:00


In [183]:
def parse_data_hibrido(texto):

    formatos = [
        "%Y-%m-%d",
        "%d/%m/%Y",
        "%Y/%m/%d",
        "%m-%d-%Y",
        "%Y%m%d"
    ]

    for f in formatos:
        try:
            return datetime.strptime(texto, f)
        except:
            pass

    data_dp = dateparser.parse(texto, languages=["pt"])
    if data_dp:
        return data_dp

    return np.nan


In [184]:
df["Data Compra"] = df["Data Compra"].astype(str)
df["Data_Tratada"] = df["Data Compra"].apply(parse_data_hibrido)

df.head(10)


,Data Compra,Data_Tratada
0,2024-05-20,2024-05-20 00:00:00
1,20/05/2024,2024-05-20 00:00:00
2,"maio 20, 2024",2024-05-20 00:00:00
3,2024/05/20,2024-05-20 00:00:00
4,05-20-2024,2024-05-20 00:00:00
5,25 fev. 2023,2023-02-25 00:00:00
6,1/1/2025 15:30:00,2025-01-01 15:30:00
7,28 de Novembro de 2023,2023-11-28 00:00:00
8,20 dez. 2024,2024-12-20 00:00:00
9,20230715,2023-07-15 00:00:00


In [185]:
df["Data Compra"] = df["Data_Tratada"]
df = df.drop(columns=["Data_Tratada"])
df.head(10)


,Data Compra
0,2024-05-20 00:00:00
1,2024-05-20 00:00:00
2,2024-05-20 00:00:00
3,2024-05-20 00:00:00
4,2024-05-20 00:00:00
5,2023-02-25 00:00:00
6,2025-01-01 15:30:00
7,2023-11-28 00:00:00
8,2024-12-20 00:00:00
9,2023-07-15 00:00:00


In [186]:
# --- Lógica para DEFINIR o Status Final ---
# 1. Defina o total de erros críticos encontrados nas suas validações
# (Usando 1, conforme o erro de data futura que você encontrou)
num_erros_criticos = 1

# 2. Determina o Status Final
status_final = "REPROVADO" if num_erros_criticos > 0 else "APROVADO"

# 3. Imprime o status de forma destacada
print("\n=======================================================")
print(f"       STATUS FINAL DA QUALIDADE DE DADOS: {status_final}       ")
print("=======================================================\n")

# --- Relatório Detalhado (String em Python) ---
relatorio_final = f"""
#######################################################
# RELATÓRIO DE QUALIDADE DE DADOS (DATETIME)
#######################################################
Projeto: Validador de Data Quality (Datas)
Data da Execução: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
Base de Dados Analisada: arq.csv
Coluna Validada: Data Compra
Total de Registros: 10

-------------------------------------------------------
## RESUMO EXECUTIVO
-------------------------------------------------------
O processo de validação de qualidade de dados resultou em um status {status_final} para a coluna "Data Compra". Foi identificado **1 registro com data no futuro** (referente ao dia 01/01/2025), o que viola a regra de negócio estabelecida.

-------------------------------------------------------
## 1. VALIDAÇÃO DE FORMATO E PADRONIZAÇÃO
-------------------------------------------------------
Objetivo: Garantir que todas as datas sejam padronizadas para o tipo datetime64[ns].
Status Pós-Tratamento: APROVADO (Todas as datas convertidas com sucesso).

-------------------------------------------------------
## 2. VALIDAÇÃO DE MISSING VALUES (Valores Ausentes)
-------------------------------------------------------
Contagem de Missing: 0 valores ausentes.
Status: APROVADO

-------------------------------------------------------
## 3. VALIDAÇÃO DE REGRAS DE NEGÓCIO
-------------------------------------------------------
- Regra: Data Antiga | Limite: Datas < 01/01/1900 | Erros: 0 | Status: APROVADO
- Regra: Data Futura | Limite: Datas > Hoje | Erros: 1 | Status: REPROVADO

-------------------------------------------------------
## RECOMENDAÇÕES FINAIS
-------------------------------------------------------
1. Correção de Dados: Isolar o registro com data futura (2025-01-01) e corrigir na fonte de informação.
2. Monitoramento: Manter a função de padronização para evitar novos erros de formato.
"""

print(relatorio_final)


       STATUS FINAL DA QUALIDADE DE DADOS: REPROVADO       


#######################################################
# RELATÓRIO DE QUALIDADE DE DADOS (DATETIME)
#######################################################
Projeto: Validador de Data Quality (Datas)
Data da Execução: 2025-11-23 22:18:52
Base de Dados Analisada: arq.csv
Coluna Validada: Data Compra
Total de Registros: 10

-------------------------------------------------------
## RESUMO EXECUTIVO
-------------------------------------------------------
O processo de validação de qualidade de dados resultou em um status REPROVADO para a coluna "Data Compra". Foi identificado **1 registro com data no futuro** (referente ao dia 01/01/2025), o que viola a regra de negócio estabelecida.

-------------------------------------------------------
## 1. VALIDAÇÃO DE FORMATO E PADRONIZAÇÃO
-------------------------------------------------------
Objetivo: Garantir que todas as datas sejam padronizadas para o tipo datetime64[ns].
Status 